In [1]:
import cv2

In [2]:
# image1 = cv2.imread('sample1.jpg', cv2.IMREAD_GRAYSCALE)
# image2 = cv2.imread('sample2.jpg', cv2.IMREAD_GRAYSCALE)

def computeMatches(image1, image2):
    minHessian = 400
    ratio_thresh = 0.7

    surf = cv2.xfeatures2d_SURF.create(hessianThreshold=minHessian)
    kp1, des1 = surf.detectAndCompute(image1, None)
    kp2, des2 = surf.detectAndCompute(image2, None)

    fl_matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
    knn_matches = fl_matcher.knnMatch(des1, des2, 2)

    matches = []
    for m,n in knn_matches:
        if m.distance < ratio_thresh * n.distance:
            matches.append(m)